Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` (you should of course delete `raise NotImplementedError()` which is there only as reminder), while not modifying the other cells (but you should run them to check the output you obtain). Please also fill your first and last name and your VU ID below:

In [1]:
STUDENT_FIRST_NAME = "Begüm" # e.g. "John" (no "J", no "J.", no "John S.M.")
STUDENT_LAST_NAME = "Yalçın"  # e.g. "Smith"
VU_ID_NUM = "2797623"          # e.g. "2789012"

---

# Assignment 3

The goal of this assignment is to make you familiar with statistical data analysis, as well as classification and regression tasks. This assignment is divided into 3 exercises, which are worth different amount of points (total is 100 points). Similar to the previous assignment, **the input and desired output are described either in the text or in the provided function docstrings.**

We assume that the folder that you work in is the one obtained by unzipping the given ``assignment3.zip`` file and thus has the following structure. When working on your submission, please respect it, otherwise the autograder will give errors.

```
assignment3.ipynb
data/
    hue/...
    emails.csv
    house_prices.csv
```

## Important remarks

- **Working together**: You are meant to work individually on the first three assignments. You can, of course, brainstorm ideas and discuss issues with your fellow students, but you are required to write your solutions individually. 
- **Plagiarism**: All your code will be automatically scanned for plagiarism. Furthermore, using the internet as a passive resource is allowed. This means that you can search for help there and partially copy code, as long as you explicitly acknowledge inside your Jupyter notebook which parts have been copied and from where. 
- **Performance**: You should optimize the computational performance of your functions. Specifically, when grading the assignments, we at times set a hard limit for each cell execution as specified in the exercise description. Function calls that take longer than that time threshold will not be awarded any points.
- **Code styling**: Your implementation will not be checked for style. However, we do encourage you to practice good code styling. See, for example, https://docs.python-guide.org/writing/style/.
- **Chronological run**: All outputs should be repeatable by doing one full “chronological” run of the notebook without any manual changes to code blocks, including parameters. (try it yourself by clicking ``Kernel -> Restart and run all``, which should give the result as handed in).
- **Handing in**: Hand in the .ipynb file of your notebook on the [Canvas page for Assignment 3](https://canvas.vu.nl/courses/68046/assignments/262963) by *Wednesday June 21st 23:59*. . 
- **Other questions**: If you have doubts/questions about the assignments, feel free to ask them in [this discussion thread](https://canvas.vu.nl/courses/68046/discussion_topics/648149) so that everyone will be able to see them and our answers. 

In [2]:
# Run this cell before anything else to import the packages you are allowed to used for this assignment

import numpy as np
import pandas as pd
import sklearn
from scipy import stats
import statsmodels.api as sm

from numpy.testing import assert_equal, assert_almost_equal

pd.options.display.max_rows = 20

# Exercise 1: Statistical data analysis (20 points)

The goal of this part of the assignment is to provide you with practice and experience in some basic data exploration and hypothesis testing with Python. You will work with data from the so-called "HUE bedtime procrastination study". A cleaned version of the data is found in (`hue_week_3.csv`), as well as another file that contains data from the post-study questionnaire that participants filled out at the end of the study (`hue_questionnaire.csv`). This file contains the following information:

| Column | Description |
-----------------------|--------------------------------------------|
| `gender`          | 1 = male, 2 = female |
| `age`           | Numeric age value | 
| `chronotype`      |    Single item (7-point scale), do you consider yourself more of a <br> morning (1) or an evening person? (7) |
| `bp_scale` | Dutch version of the Bedtime Procrastination Scale |
| `motivation` | Questions pertaining to personality traits related to procrastination. <br> Single item (7-point scale), how motivated were you to go to bed on <br> time each night? (1 = not motivated, 7 = very motivated) |
| `daytime_sleepiness` | Dutch translation of the Epworth Sleepiness Scale <br> (4-point scale from 0-3; 8 questions, values summed) |
| `self_reported_effectiveness` | Single item (7-point scale), <br> do you feel more rested since the intervention |

In this part of the assignment, you will use Python to examine the post-questionnaire data in relation to the HUE data file, visualize trends and relationships, look for correlations between factors, test for significant differences between groups and build a regression model to predict bedtime delay. In order to perform the analyses, a number of transformations on the data still need to be done.

## Importing and cleaning the file (0 points)

The read-only cell below already implements the ``read_data`` function, which returns a clean dataframe. The precise steps that the function takes are detailed as follows: 

- Reads the HUE data file and the questionnaire data file into two separate pandas DataFrames.

- Creates a new DataFrame that contains the following Series:
    
| Column | Description |
-----------------------|--------------------------------------------|
| `ID` | Participant ID |
| `group` | Participant group (1 for experimental, 0 for control) |
| `delay_nights` | The number of nights the participant delayed their bedtime (range: 0-12) |
| `delay_time` | The mean time in seconds a participant delayed their bedtime <br> (total delay in seconds, divided by the number of observations <br> measured for each individual, rounded to nearest second) |
| `sleep_time` | The mean in-bed time in seconds of a participant (rounded to nearest second) <br> Average should be taken over the ``In Bed`` columns. Disregard inconsistencies with the difference ``Rise Time`` - ``Bed Time``  |
    
    
- Sets the index of this new DataFrame to `ID`. Note that there should only be a single row per participant ID.    

- Fills this new DataFrame by transforming data from the DataFrame about participants' bedtimes (from the HUE data file).

- Merges this new DataFrame with the post-questionnaire data and store the resulting DataFrame in a new variable. Perform this merging operation of the two DataFrames in such a way that the resulting Data Frame only contains IDs that were present in both datasets.

- Removes the rows that have NaN values in this merged DataFrame.

In [3]:
def read_data(sleepdatafile, surveydatafile):
    
    def calculate_number_and_time_delay(row):
        numberDelayNights = 0
        sumDelayTime = 0
        meanDelayTime = 0
        isNan = True

        i = 4
        while i <= 114:
            if not np.isnan(row[i]):
                isNan = False
                if float(row[i]) >= 0:
                    numberDelayNights += 1
                    sumDelayTime = sumDelayTime + float(row[i])
            i = i + 10
        if numberDelayNights > 0:
            meanDelayTime = round(sumDelayTime / numberDelayNights, 0)
        elif isNan:
            meanDelayTime = np.nan
            numberDelayNights = np.nan
        return numberDelayNights, meanDelayTime


    def create_new_dataframe(sleepDf):
        df = pd.DataFrame(columns=['group', 'delay_nights', 'delay_time'])
        dfs_to_concat = []

        for index, row in sleepDf.iterrows():
            ID = index
            group = row[0]
            numberDelayNights, meanDelayTime = calculate_number_and_time_delay(row)
            new_row = pd.Series({'group': group, 'delay_nights': numberDelayNights, 'delay_time': meanDelayTime}, name=ID)
            dfs_to_concat.append(new_row)

        df = pd.concat(dfs_to_concat, axis=1).T

        return df


    def merge_dataFrames(transformedDf, surveyDf):
        return pd.merge(transformedDf, surveyDf, how = 'inner', left_index = True, right_index = True)


    def calculate_sleep_duration(sleepDf, mergedDf):
        mergedDf["sleep_duration"] = np.nan

        for index,row in sleepDf.iterrows():
            numberNights = 0
            sumTime = 0
            meanTime = 0
            isNan = True

            if index in mergedDf.index:
                i = 7
                while i <= 117:
                    if not np.isnan(row[i]):
                        isNan = False
                        numberNights += 1
                        sumTime = sumTime + float(row[i])
                    i = i + 10

                if numberNights > 0:
                    meanTime = round(sumTime / numberNights, 0)
                elif isNan:
                    meanTime = np.nan
                mergedDf.at[index, 'sleep_duration']=meanTime

    sleepDf = pd.read_csv(sleepdatafile, delimiter = ',', index_col = 0)
    surveyDf = pd.read_csv(surveydatafile, delimiter = ',', index_col = 0)
    
    transformedDf = create_new_dataframe(sleepDf)
    mergedDf = merge_dataFrames(transformedDf, surveyDf)
    calculate_sleep_duration(sleepDf, mergedDf)
    mergedDfNoNan = mergedDf.dropna()
    mergedDfNoNan.loc[:, 'group'] = mergedDfNoNan['group'].astype('int64')
    
    return mergedDfNoNan

In [4]:
# Run this cell to correctly import the dataframe (and ignore the warnings)

sleepdatafile   = 'data/hue/hue_week_3.csv'
surveydatafile  = 'data/hue/hue_questionnaire.csv'
mergedDf = read_data(sleepdatafile, surveydatafile)
display(mergedDf)

/var/folders/0m/4f8p309n38dgcm6s7hjkbtx00000gn/T/ipykernel_12746/2107794122.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mergedDfNoNan.loc[:, 'group'] = mergedDfNoNan['group'].astype('int64')


,group,delay_nights,delay_time,gender,age,chronotype,bp_scale,motivation,daytime_sleepiness,self_reported_effectiveness,sleep_duration
1,0,10.0,6030.0,2,20,7,6.11,4,17,4,31193.0
2,0,7.0,2494.0,2,25,5,5.22,4,21,2,27852.0
4,1,7.0,4149.0,2,30,1,6.67,5,14,5,30343.0
5,1,9.0,2620.0,2,27,5,2.67,6,12,6,32573.0
6,0,7.0,5417.0,1,27,6,5.11,6,14,3,29822.0
...,...,...,...,...,...,...,...,...,...,...,...
52,1,9.0,1947.0,1,18,7,5.56,3,11,3,29001.0
55,0,5.0,3672.0,1,26,4,5.00,2,15,1,26984.0
58,0,11.0,3622.0,1,35,7,6.33,6,12,2,28370.0
61,0,9.0,5707.0,2,39,7,5.89,5,19,1,30725.0


## Correlation coefficients (5 points)

Use the `scipy.stats` package and, respectively, the Pearson correlation test and the Kendall rank correlation test, to calculate the following correlation coefficients for the dataframe ``mergedDf`` defined above:

- the Pearson correlation coefficient between bedtime procrastination scale ( `bp_scale`, a personality trait) and mean time spent delaying bedtime,    

- the Pearson correlation coefficient between mean time spent delaying bedtime and daytime sleepiness,

- the Kendall rank correlation coefficient between age and mean time spent delaying bedtime.

Save them into the variables `r1`, `r2`, `tau` without any rounding. Save also the respective p-values into the variables `pvalue1`, `pvalue2`, `pvalue3` (also without any rounding).

In [5]:
def calculate_correlations(mergedDf):
    r1, pvalue1 = stats.pearsonr(mergedDf['bp_scale'], mergedDf['delay_time'])
    r2, pvalue2 = stats.pearsonr(mergedDf['delay_time'], mergedDf['daytime_sleepiness'])
    tau, pvalue3 = stats.kendalltau(mergedDf['age'], mergedDf['delay_time'])
    return r1, pvalue1, r2, pvalue2, tau, pvalue3
    
# YOUR CODE HERE
#raise NotImplementedError()

In [6]:
r1, pvalue1, r2, pvalue2, tau, pvalue3 = calculate_correlations(mergedDf)

statistics = [r1,r2,tau]
pvalues = [pvalue1, pvalue2, pvalue3]

print("Correlation tests:\n")
for (statistic, pvalue) in zip(statistics, pvalues):
    print('The value of the test statistic is:',statistic)
    print('The p-value is:', pvalue,'\n')


Correlation tests:

The value of the test statistic is: 0.5926593048409182
The p-value is: 8.837285991335382e-05 

The value of the test statistic is: 0.0821236747349586
The p-value is: 0.6240188835001698 

The value of the test statistic is: -0.08238035084305242
The p-value is: 0.4725232598663073 



## Significant differences (5 points)

We keep analyzing the dataframe ``mergedDf`` defined above. Use the `scipy.stats` package to determine whether there are significant differences (at 5\% significance level) between the experimental group and the control group in terms of:
<br></li>
<li>
    the time participants spent in bed each night,   
<br></li>
<li>
    the number of nights participants delayed their bedtime,
<br></li>
<li>
    the mean time participants spent delaying their bedtime.
</li>
</ul> 

Use the t-test or the Wilcoxon rank-sum test to reach a conclusion and use knowledge gained in the courses Statistics and Statistical Data Analysis to determine which statistical test is appropriate. Save the conclusions - either the string `'significant difference'` or `'no significant difference'` - into the variables `diff1`, `diff2`, `diff3`.

\* Note that in this exercise you are not expected to explicitly motivate the choice of an appropriate test, but you will be in the final project.

In [7]:
def perform_tests(mergedDf):
    control_group = mergedDf[mergedDf['group']==0]
    experimental_group = mergedDf[mergedDf['group']==1]
    stat1, pvalue1 = stats.ttest_ind(experimental_group['sleep_duration'], control_group['sleep_duration'])
    difference_1 = 'significant difference' if pvalue1 <=0.05 else 'no significant difference'
    stat2, pvalue2 = stats.ranksums(experimental_group['delay_nights'], control_group['delay_nights'])
    difference_2 = 'significant difference' if pvalue2 <=0.05 else 'no significant difference'
    stat3, pvalue3 = stats.ranksums(experimental_group['delay_time'], control_group['delay_time'])
    difference_3 = 'significant difference' if pvalue3 <=0.05 else 'no significant difference'
    
    return difference_1, difference_2, difference_3
# YOUR CODE HERE
#raise NotImplementedError()

In [8]:
diff1, diff2, diff3 = perform_tests(mergedDf) 

print('The time participants spent in bed each night:', diff1)
print('The number of nights participants delayed their bedtime:', diff2)
print('The mean time participants spent delaying their bedtime:', diff3)


The time participants spent in bed each night: no significant difference
The number of nights participants delayed their bedtime: no significant difference
The mean time participants spent delaying their bedtime: significant difference


## Ordinary Least Squares regression (10 points)

Also for this last question, you will use the dataframe ``mergedDf`` defined above. Use `statsmodels.api` to build a (OLS) regression model for `delay_time` on the predictors `age`, `bp_scale`, and  `chronotype` (in this order!). 

Return the coefficients and the conclusion on significance of the model. More specifically, into the variable `parameters` directly save the output of `*.params` applied to the model (without any rounding), and into the variable `conclusion` save the string 'significant' or 'not significant'.

\* Convince yourself that the basic diagnostics for this model are OK. Note that here you are not expected to explicitly check the diagnostics, but you will be in the final project.

In [9]:
def regression_analysis(mergedDf):
    predictors_list = ['age', 'bp_scale','chronotype']
    X=sm.add_constant(mergedDf[predictors_list])
    y= mergedDf['delay_time']
    model = sm.OLS(y, X).fit()
    params= model.params
    conclusion = 'significant' if model.f_pvalue <= 0.05 else 'not_significant'
    return params, conclusion
# YOUR CODE HERE
#raise NotImplementedError()

In [10]:
parameters, conclusion = regression_analysis(mergedDf)

print('The parameters of the model are:')
print(parameters)
print('\nThe model is', conclusion)


The parameters of the model are:
const        -1399.595480
age            -25.178161
bp_scale       882.064704
chronotype      99.552661
dtype: float64

The model is significant


# Exercise 2: Classification SPAM vs. HAM (40 points)

The goal of this exercise is to build a classifier that is able to recognize spam email. "Ham" is e-mail that is not spam. The email data is provided in the following dataframe:

In [11]:
df = pd.read_csv('data/emails.csv')
df.head()

,text,label
0,Subject: anshuman shrivastava\n sandeep : vinc...,0
1,Subject: asset sales chart\n please see the at...,0
2,">>>>> ""G"" == Gordon Mohr writes: G> Gary L...",0
3,Subject: stockmarket smail - cap profiie\n bis...,1
4,"Subject: yukos oil\n dear friend ,\n i am mr o...",1


The data contains two columns. The `text` column contains the email content and `label` contains the corresponding labels, i.e., whether the email is spam (1) or ham (0). In the following, we will use the variables `X` and `y` to denote the text features matrix and target vector, respectively.

The goal is to implement any model of your choice that predicts whether a message is spam or ham.

- You are free to choose any model, including random forests, support vector machines, etc. 
- Your model will be evaluated on a hidden test that is not included in the given data set, but shares the same characteristics. Therefore, if you want to obtain a high score on this hidden test set, you have to make sure to follow the best practices in training your model to prevent underfitting and overfitting.
- The hidden test contains 1498 entries.
- Your score is determined based on the F1 score on the hidden test set. To get non-zero points, your model should get an F1-score of at least **95%** on the hidden test. To get the maximum amount of points, your model should get an F1-score of **99%** or higher.
- The training of your model should take no longer than **45 seconds**.

In [12]:
X = df['text'].values
y = df['label'].values

In [25]:
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB


def best_spamham(X, y):
    pipeline= make_pipeline(CountVectorizer(), MultinomialNB())
    pipeline.fit(X,y)
    return pipeline
    
# YOUR CODE HERE
#raise NotImplementedError()

In [26]:
# No test available; the code shows how your model will be evaluated

from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import time

t0 = time.perf_counter()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
best = best_spamham(X_train, y_train)
predicted = best.predict(X_test)
t1 = time.perf_counter()
print(f'Your model took {t1-t0:.4} seconds to be trained.\n')

# The F1 score will be used to evaluate your score
test_score = f1_score(y_test, predicted)
print(f"Evaluation for function best_spamham. Your F1 score is {test_score:.4} on the public test set.\n")


Your model took 2.796 seconds to be trained.

Evaluation for function best_spamham. Your F1 score is 0.9749 on the public test set.



# Exercise 3: Regression on Californian house prices (40 points)

The goal of this exercise to apply regression to predict house prices in California. Here is a brief description of the data set:

```
California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived from the 1990 U.S. census, using one row per census
block group. A block group is the smallest geographical unit for which the U.S.
Census Bureau publishes sample data (a block group typically has a population
of 600 to 3,000 people).

An household is a group of people residing within a home. Since the average
number of rooms and bedrooms in this dataset are provided per household, these
columns may take surpinsingly large values for block groups with few households
and many empty houses, such as vacation resorts.
```

Here is the dataframe of the housing prices.

In [27]:
housing = pd.read_csv('data/house_prices.csv', index_col=0)
housing.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
13646,1.9234,43.0,4.821023,1.099432,1181.0,3.355114,34.08,-117.31,0.746
163,3.2250,46.0,4.326640,1.117805,1373.0,1.838019,37.81,-122.25,2.188
18240,2.5208,52.0,3.908163,1.035714,448.0,2.285714,37.40,-122.08,3.167
12061,5.1929,27.0,6.424460,1.032374,939.0,3.377698,33.87,-117.57,1.650
14177,3.0446,32.0,4.895075,1.002141,1741.0,3.728051,32.72,-117.08,1.019


`X` contains the features and `y` contains the target values.

In [28]:
X = housing.drop(['MedHouseVal'], axis=1).values
y = housing['MedHouseVal'].values

The goal is to implement any model of your choice that predicts the house prices.
- You are free to choose any model for this exercise.
- Your model will be evaluated on a hidden test that is not included in the given data set, but shares the same characteristics. Therefore, to ensure a high score on this hidden test set, you should make sure to follow the best practices in validating your model (i.e., train, validation and test set) to avoid underfitting and overfitting.
- The hidden test set contains 1640 entries.
- Your score is determined based on the root mean square error (RMSE) score evaluated on the hidden test set. To get non-zero points, your model should obtain an RMSE lower than **0.7** on the hidden test. To get the maximum number of points, your model should obtain an RMSE of at most **0.29**.
- The training of your model should take no longer than **45 seconds**.

In [29]:
from sklearn.metrics import mean_squared_error

def rmse(target, predicted):
    """
    Return the root mean squared error between the target and predicted vector.
    """
    return mean_squared_error(target, predicted, squared=False)

In [36]:
from sklearn.ensemble import RandomForestRegressor

def best_model(X, y):

    model=RandomForestRegressor()
    model.fit(X,y)
    return model

# YOUR CODE HERE
#raise NotImplementedError()

In [37]:
# No test available; the code shows how your model will be evaluated

import time

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

t0 = time.perf_counter()
best = best_model(X_train, y_train)
predicted = best.predict(X_test)
t1 = time.perf_counter()
print(f'Your model took {t1-t0:.4} seconds to be trained.\n')

# The RMSE will be used to evaluate your score
result = rmse(y_test, predicted)
print(f'Your RMSE score on public test set is {result:.4}.')


Your model took 12.17 seconds to be trained.

Your RMSE score on public test set is 0.4956.
